## 1. Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [10]:
df = pd.read_csv('users_behavior.csv')

Изучение датасета

In [15]:
df.shape

(3214, 5)

In [13]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [14]:
df.sample()

,calls,minutes,messages,mb_used,is_ultra
3098,46.0,360.78,12.0,12706.94,1


In [35]:
df.tail()

,calls,minutes,messages,mb_used,is_ultra
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0
3213,80.0,566.09,6.0,29480.52,1


Пропусков нет

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [17]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Наблюдается дисбланс классов. Пользователей тарифа Ультра меньше в 2 раза.

Создаю переменные для признаков и целевого признака

In [19]:
X = df.drop('is_ultra', axis=1)
y = df['is_ultra']

## 2. Разбейте данные на выборки

Так как у нас нет скрытой тестовой выборки, то с помощью `train_test_split` разбиваю данные на две части - обучающую и тестовую в соотношении 70:30 соответственно. Дисбаланс классов сохраняем.

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

## 3. Исследуйте модели

#### Логистическая регрессия

Перед нами стоит задача классификации, поэтому использую логистическую регрессию. Значимым гиперпараметром является коэффициент регуляризации `C`. Для поиска оптимального значения использую `GridSearchCV` с кросс-валидацией на пяти подвыборках.

In [23]:
parameters = {'C': (0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000)}

In [27]:
grid_lr = GridSearchCV(LogisticRegression(random_state=42), parameters, cv=5)
grid_lr.fit(X_train, y_train)
params = grid_lr.best_params_
lr = LogisticRegression(**params, random_state=42)
lr.fit(X_train, y_train)
grid_lr.best_params_

{'C': 10}

#### Решающее дерево и случайный лес

Для `DecisionTreeClassifier` и `RandomForestClassifier` поиск будет осуществляться по гиперпараметрам `max_depth`, `min_samples_split` и `min_samples_leaf`. Создаю переменную `tree_params` c различными вариантами значений указанных гиперпараметров.

In [25]:
tree_params = {'max_depth': list(range(2, 11)),
               'min_samples_split': list(range(2, 21, 2)),
               'min_samples_leaf': list(range(1, 21, 2))}

Выбор гиперпараметров для решающего дерева. Для поиска оптимального значения использую `GridSearchCV` с кросс-валидацией на пяти подвыборках.

In [28]:
grid_dtc = GridSearchCV(DecisionTreeClassifier(random_state=42),
                         tree_params, cv=5, scoring='accuracy') 
grid_dtc.fit(X_train, y_train)
params = grid_dtc.best_params_
dtc = DecisionTreeClassifier(**params, random_state = 42)
dtc.fit(X_train, y_train)
grid_dtc.best_params_

{'max_depth': 7, 'min_samples_leaf': 13, 'min_samples_split': 2}

Выбор гиперпараметров для случайного леса. Для поиска оптимального значения использую `GridSearchCV` с кросс-валидацией на пяти подвыборках.

In [29]:
grid_rfc= GridSearchCV(RandomForestClassifier(random_state=42), 
                         tree_params, cv=5, scoring='accuracy') 
grid_rfc.fit(X_train, y_train)
params = grid_rfc.best_params_
rfc = RandomForestClassifier(**params, random_state = 42)
rfc.fit(X_train, y_train)
grid_rfc.best_params_

{'max_depth': 8, 'min_samples_leaf': 3, 'min_samples_split': 12}

## 4. Проверьте модель на тестовой выборке

Функция для расчета `accuracy` предсказания на тестовой выборке после обучения модели на тренировочной выборке.

In [30]:
def test_score(model):
    test_predictions = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, test_predictions)
    print(test_accuracy)

In [31]:
test_score(lr)

0.6984455958549223


In [32]:
test_score(dtc)

0.8041450777202073


In [33]:
test_score(rfc)

0.8124352331606217


### Вывод
- На тестовой выборке лучшее значение accuracy 0.81 удалось достичь, используя RandomForestClassifier. Результат DecisionTreeClassifier чуть меньше - 0.8. Самый худший результат 0.7 показала LogisticRegression.

## 5. (бонус) Проверьте модели на адекватность

Предположим, что в новых данных сохраняется тот же дисбаланс классов, как в исходной выборке. Найдем долю пользователей тарифа Смарт.

In [34]:
(df['is_ultra']==0).sum() / df.shape[0]

0.693528313627878

Таким образом качество предсказаний моделей выше, чем наше предположение о сохранении дисбаланса классов в тех же пропорциях.